In [19]:
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
from scipy.stats import linregress
from influenza_USA.NC_forecasts.utils import get_NC_influenza_data, compute_WIS, simulate_geometric_random_walk, get_historic_drift

In [ ]:
# settings
start_baseline_month = 12 # expressed as Hubverse reference date
start_baseline_day = 1
end_baseline_month = 4
end_baseline_day = 7
drift_horizon = 4
sigma = 0.375
seasons = ['2014-2015','2015-2016', '2016-2017', '2017-2018', '2018-2019', '2019-2020', '2023-2024']

In [34]:
# LOOP seasons
collect_seasons=[]
for focal_season in seasons:
    ## get the current season's data
    data = 7*get_NC_influenza_data(datetime(int(focal_season[0:4]), start_baseline_month, start_baseline_day) - timedelta(weeks=1),
                                    datetime(int(focal_season[0:4])+1, end_baseline_month, end_baseline_day)+timedelta(weeks=4),
                                    focal_season)['H_inc']
    ## LOOP weeks
    collect_weeks=[]
    for date in data.index[:-4]:
        ### GET historical drift 
        mu, _ = get_historic_drift(focal_season, seasons, date, drift_horizon)
        ### SIMULATE baseline model
        simout = simulate_geometric_random_walk(mu, 0.375, date, data[date], n_sim=1000, n_weeks=4)
        ### COMPUTE WIS score
        collect_weeks.append(compute_WIS(simout, data))
    ## CONCATENATE WEEKS
    collect_weeks = pd.concat(collect_weeks, axis=0)
    collect_weeks = collect_weeks.reset_index()
    collect_weeks['season'] = focal_season
    collect_seasons.append(collect_weeks)
# CONCATENATE SEASONS
collect_seasons = pd.concat(collect_seasons, axis=0)
print(np.mean(collect_seasons.groupby(by=['season'])['WIS'].mean()))
print(np.sum(collect_seasons.groupby(by=['season'])['WIS'].mean()))

80.17036552793942
561.1925586955759
